<a href="https://colab.research.google.com/github/Amitgm/Grip-Computer-Vision/blob/social_distancing/Social_distancing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Name - Amit George**

**Task 3 - Social Distancing detector** 

**Grip@ The Spark foundation network**

**Objective- To detect social distancing between people within a certain crowd**

**Load the necessary libraries**

In [ ]:
import os
import matplotlib.pyplot as plt
import cv2
from natsort import natsorted
from PIL import Image
import xml.etree.ElementTree as ET
import glob
import json
import shutil
from bs4 import BeautifulSoup
import torchvision
from torchvision import transforms, datasets, models
from IPython.display import HTML
from base64 import b64encode

**Loading the Model for detecting Person's**

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5x',
                       pretrained=True, verbose=False)
#model.cuda('cuda:0');

In [ ]:
# loading video-files from drive
from google.colab import drive
drive.mount('/content/drive')

In [69]:
input_dir='/content/drive/MyDrive/video-files'
images = natsorted(glob.glob(os.path.join(input_dir, '*.jpg')))

**Obtaining the bounding box coordinates of detected person's from the images** 

In [70]:
dataset=[]
for img in images:
 image=cv2.imread(img)
 image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
 results = model([image[:, :, ::-1]])  # Pass the frame through the model and get the boxes
 results=results.xyxy[0].numpy()
 reults=results[results[:,5]==0] # only people
 data=results[:,:4] # take the first 4 bounding box corrdiates of pixels.
 dataset.append(data)

**Calculate the centre distance between two persons's in the image**

In [42]:
def center_distance(xyxy1, xyxy2):
    '''Calculate the distance of the centers of the boxes.'''
    a, b, c, d = xyxy1
    x1 = int(np.mean([a, c]))
    y1 = int(np.mean([b, d]))

    e, f, g, h = xyxy2
    x2 = int(np.mean([e, g]))
    y2 = int(np.mean([f, h]))
    
    dist = np.linalg.norm([x1 - x2, y1 - y2])
    return dist, x1, y1, x2, y2

**Calculate the social distance between people in the image, the red box means the person is close to the other person within 100 units and green box when far away from each other**

In [82]:
image_list=[]
distance=100
for data,img in zip(dataset,images):
 image=cv2.imread(img)
 image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
 colors = ['green']*len(data)
 for i in range(len(data)):
        for j in range(i+1, len(data)):
            # Calculate distance of the centers
            dist, x1, y1, x2, y2 = center_distance(data[i], data[j])
            #distances.append(dist)
            if dist < distance:
                #distances.append(dist)
                # If dist < distance, boxes are red and a line is drawn
                colors[i] = 'red'
                colors[j] = 'red'
                image = cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)

            for i, (x1, y1, x2, y2) in enumerate(data):
                  # Draw the boxes
                if colors[i] == 'green':
                  color = (0, 255, 0)
                else:
                  color = (255, 0, 0)
                image = cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
 image_list.append(image)
        


**Display the video of social distancing**

In [6]:
def display_video(path):
    '''Display video in Colab.'''
    #compressed_path = path.split('.')[0]
    #compressed_path = 'compressed_' + compressed_path + '.mp4'

    if os.path.exists(path):
        os.remove(path)

    # Convert video
    os.system(f"ffmpeg -i {path} -vcodec libx264 {path}")

    # Show video
    mp4 = open(path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

In [ ]:
!mv '/content/drive/MyDrive/social_distance.mp4' '/content/'

In [ ]:
path2='/content/'
display_video(path2)

**Save the object detection images to be later converted to video**

In [96]:
from PIL import Image
for i,j in zip(image_list,images):
  im = Image.fromarray(i)
  im.save("/content/drive/MyDrive/social_distancing/{}.jpeg".format(os.path.basename(j)))

In [ ]:
# zip file for the data of images to be converted to video
!zip -r '/content/social_distancing.zip' '/content/drive/MyDrive/social_distancing' 